In [1]:
# Création du dossier WebCrawler contenant l'ensemble des fichiers utiles au fonctionnement de scrapy
!scrapy startproject MangaCrawler

Error: scrapy.cfg already exists in /home/yannis/Projects/Ecole_IPSSI/2022-2023/WebScraping/Scrapy/WebScraping/MangaCrawler


In [9]:
# création du projet Manga
!cd MangaCrawler && scrapy genspider mangascantrad https://manga-scantrad.net/

Created spider 'mangascantrad' using template 'basic' in module:
  MangaCrawler.spiders.mangascantrad


In [ ]:
# Ajouter au fichier items.py & Liste des données qui seront récupérés par manga
class MangacrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    img = scrapy.Field()
    rating = scrapy.Field()
    last_chapter = scrapy.Field()
    link = scrapy.Field()
    genres = scrapy.Field()
    published_date = scrapy.Field()
    state = scrapy.Field()
    nb_comments = scrapy.Field()
    pass

In [ ]:
# La class MangascantradSpider
import scrapy
from scrapy import Request
from MangaCrawler.items import MangaGenres, MangacrawlerItem, DataBase
import sqlalchemy as db
from sqlalchemy.orm import declarative_base, relationship

class MangascantradSpider(scrapy.Spider):
    name = 'mangascantrad'
    allowed_domains = ['manga-scantrad.net']

    # Liste des urls par pages
    start_urls = [f'http://manga-scantrad.net/manga/page/{n}' for n in range(1, 31)]

    # Création de la base de données
    database = DataBase('database_manga')

    # Creation des tables avec une relation ManyToMany
    Base = declarative_base()
    association_table = db.Table(
        "mangas_assoc_genres",
        Base.metadata,
        db.Column("mangas_id", db.ForeignKey("mangas.id_")),
        db.Column("mangas_genres_id", db.ForeignKey("mangas_genres.id_")),
    )
    database.create_table('mangas_genres',
                          id_=db.Integer,
                          name=db.String,
                          )

    database.create_table('mangas',
                          id_=db.Integer,
                          title=db.String,
                          img=db.String,
                          rating=db.String,
                          last_chapter=db.String,
                          link=db.String,
                          genres=db.String,
                          published_date=db.String,
                          state=db.String,
                          nb_comments=db.String,
                          children=relationship("mangas_genres", secondary=association_table)
                          )

    def start_requests(self):
        print("iok")
        # for url in self.start_urls:
        #     yield Request(url=url, callback=self.parse_manga)

    def parse_manga(self, response):
        mangas = response.css('div.js-categories-seasonal.js-block-list.list table tr')[1:]
        for manga in mangas:
            item = MangacrawlerItem()

            # Nom manga
            try:
                item['name'] = manga.css('td')[1].css('a.hoverinfo_trigger.fw-b strong::text').get()
            except:
                item['name'] = 'None'

            # Image manga
            try:
                item['img'] = manga.css('td')[0].css('a.hoverinfo_trigger img').attrib['data-src']
            except:
                item['img'] = 'None'

            # Description manga
            try:
                item['description'] = manga.css('td')[1].css('div.pt4::text').get()
            except:
                item['description'] = 'None'

            # Ajouter dans la base de données
            self.database.add_row('manga',
                                  name=item['name'],
                                  img=item['img'],
                                  description=item['description']
                                  )
            yield item



In [9]:
# Exécution de scrapy
!cd MangaCrawler/MangaCrawler/spiders && scrapy crawl mangascantrad -o mangascantrad.csv

Traceback (most recent call last):
  File "/home/yannis/.local/lib/python3.9/site-packages/sqlalchemy/sql/schema.py", line 134, in _init_items
    spwd = item._set_parent_with_dispatch
  File "/home/yannis/.local/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py", line 1244, in __getattr__
    return self._fallback_getattr(key)
  File "/home/yannis/.local/lib/python3.9/site-packages/sqlalchemy/util/langhelpers.py", line 1218, in _fallback_getattr
    raise AttributeError(key)
AttributeError: _set_parent_with_dispatch

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/yannis/.local/bin/scrapy", line 8, in <module>
    sys.exit(execute())
  File "/home/yannis/.local/lib/python3.9/site-packages/scrapy/cmdline.py", line 153, in execute
    cmd.crawler_process = CrawlerProcess(settings)
  File "/home/yannis/.local/lib/python3.9/site-packages/scrapy/crawler.py", line 304, in __init__
    super().__init__(settings)
